# Example: Let's build a ternary price tree
In this example, students will become familiar with the different internal representations of trees, particularly a ternary price tree where the price of a commodity tomorrow can go up, stay the same, or go down. 
* Array-based representations encode the `data` of the tree and do not encode any information about the connectivity of the tree, i.e., there is no information about `parent=>child` relationships. The creator of the tree is expected to know this information
* On the other hand, adjacency-based tree representations encode both the `data` of the tree and information about the `connectivity` of the tree, i.e., information about `parent=>child` relationships. 

## Setup
This example may use external third-party packages. In the `Include.jl` file, we load our codes to access them in the notebook, set some required paths for this example, and load any required external packages.

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-6/L6c`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-6/L6c/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-6/L6c/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-6/L6c/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-6/L6c/Manifest.toml`


## Example 1: Array-based representation of a `3-tree`
Let's consider the array-based representation of a `3-tree` where each `node` in the tree has `three` children nodes. Each parent `node` is a hypothetical price at time $t$, while each `child` node is a possible price at time $t+\Delta{t}$.
* We've modeled this case using the `ArrayBasedTernaryCommodityPriceTree` type, encoded in the `src/Types.jl` file. We pass some required information to a `build(...)` method encoded in the `src/Factory.jl` file to build this type.

In [2]:
array_price_tree = build(ArrayBasedTernaryCommodityPriceTree;
    h = 2, price = 100.0, u = 0.04, d = 0.01, k = 3)

ArrayBasedTernaryCommodityPriceTree(Dict(5 => 104.0, 12 => 98.01, 8 => 100.0, 1 => 104.0, 0 => 100.0, 6 => 102.96, 11 => 99.0, 9 => 99.0, 3 => 99.0, 7 => 104.0…), 2, 3)

In [3]:
array_price_tree.data

Dict{Int64, Float64} with 13 entries:
  5  => 104.0
  12 => 98.01
  8  => 100.0
  1  => 104.0
  0  => 100.0
  6  => 102.96
  11 => 99.0
  9  => 99.0
  3  => 99.0
  7  => 104.0
  4  => 108.16
  2  => 100.0
  10 => 102.96

## Example 2: Adjacency-based representation of a `3-tree`
Alternatively, we could consider an adjacency-based representation of the price tree that breaks apart the data representation for the connectivity information. 
* We've modeled this case using the `AdjacencyBasedTernaryCommodityPriceTree` type encoded in the `src/Types.jl` file. We pass some required information to a `build(...)` method encoded in the `src/Factory.jl` file to build this type.

In [4]:
adj_price_tree = build(AdjacencyBasedTernaryCommodityPriceTree;
    h = 2, price = 100.0, u = 0.04, d = 0.01, k = 3)

AdjacencyBasedTernaryCommodityPriceTree(Dict(5 => 104.0, 12 => 98.01, 8 => 100.0, 1 => 104.0, 0 => 100.0, 6 => 102.96, 11 => 99.0, 9 => 99.0, 3 => 99.0, 7 => 104.0…), Dict(0 => [1, 2, 3], 2 => [7, 8, 9], 3 => [10, 11, 12], 1 => [4, 5, 6]), 2, 3)

In [5]:
adj_price_tree.data

Dict{Int64, Float64} with 13 entries:
  5  => 104.0
  12 => 98.01
  8  => 100.0
  1  => 104.0
  0  => 100.0
  6  => 102.96
  11 => 99.0
  9  => 99.0
  3  => 99.0
  7  => 104.0
  4  => 108.16
  2  => 100.0
  10 => 102.96

In [6]:
adj_price_tree.connectivity

Dict{Int64, Vector{Int64}} with 4 entries:
  0 => [1, 2, 3]
  2 => [7, 8, 9]
  3 => [10, 11, 12]
  1 => [4, 5, 6]

## Check: Are our `3-tree` indexing and dimensions correct?
For a __full__ `k-ary` tree with no replacement, the indices of the children of node $i$, denoted by the set $\mathcal{C}_{i}$, are given by:
$$
\begin{equation}
\mathcal{C}_{i}=\left\{k\cdot{i}+1,k\cdot{i}+2,\dots,k\cdot{i}+k\right\}
\end{equation}
$$
and the total number of nodes of tree $\mathcal{T}$, denoted by $N_{h}$, is given by:
$$
\begin{equation}
N_{h} = \sum_{j=0}^{h}k^j = \frac{k^{h+1}-1}{k-1}
\end{equation}
$$
where $N_{h}$ includes the final layer of leaves. Let's check these equations by looking at the `adj_price_tree` model. First, let's specify what `k` and `h` are:

In [7]:
h = adj_price_tree.h;
k = adj_price_tree.k;

Next, let's use the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to test the dimensions of our `3-tree` implementation versus the theoretical expressions shown in the lecture (and above).
* First, let's test the tree's overall size, i.e., the number of elements $N_{h}$, versus our implementation. If the values do not agree, an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) will be thrown.

In [8]:
@assert (k^(h+1)-1)/(k-1) == length(adj_price_tree.data) 

Next, let's check the indexing of the children of each node in the tree, i.e., check the $\mathcal{C}_{i}$ set for each node $i$ in the `adj_price_tree` model.
* We'll do this by iterating over the `connectivity` dictionary and computing the child index set $\mathcal{C}_{i}$ for each `key` in the `connectivity` dictionary. If the $\mathcal{C}_{i}$ sets are not the same, an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) will be thrown.

In [9]:
for (i,Cᵢ) ∈ adj_price_tree.connectivity
    
    @show (i,Cᵢ)

    # compute what we believe the child set should be -
    C̄ᵢ = [k*i+j for j = 1:k] # hmmm: https://docs.julialang.org/en/v1/manual/arrays/#man-comprehensions

    # compare -
    result = (@assert Cᵢ == C̄ᵢ)
    println("Children of node $(i) are correct")
end

(i, Cᵢ) = (0, [1, 2, 3])
Children of node 0 are correct
(i, Cᵢ) = (2, [7, 8, 9])
Children of node 2 are correct
(i, Cᵢ) = (3, [10, 11, 12])
Children of node 3 are correct
(i, Cᵢ) = (1, [4, 5, 6])
Children of node 1 are correct
